<a href="https://colab.research.google.com/github/Aiswaryadevi/AI-deeplearning/blob/master/Forest_Cover_classification_with_EarlyStopping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Load the dataset from sklearn
from sklearn.datasets import fetch_covtype
# Importing both TensorFlow and its high level API - Keras.
import tensorflow as tf
from tensorflow import keras
# Setting the random seeds for repeatability
tf.random.set_random_seed(42)
np.random.seed(42)


In [3]:
# When return_X_y = True, the load function
# return data and target instead of Bunch object.
X, y = fetch_covtype(return_X_y=True)
print(type(X))


<class 'numpy.ndarray'>


In [4]:
# Reduce the number of attributes, consider only first 10 attributes.
X_10 = X[:,:10]
print(X_10.shape)

(581012, 10)


In [5]:
# Split the data into 90% training and 10% testing
from sklearn.model_selection import train_test_split
# The 10% testing data obtained during this split will be take as our entire database.
# This is because the original dataset is too big.
X10_train, X10_test, y10_train, y10_test = train_test_split(X_10, y, test_size=0.1,
 stratify=y, random_state=42)
print(X10_test.shape)

(58102, 10)


In [6]:
# Handle only the modified 1% dataset. Split that into training and testing.
# X and y are updated with the downsized dataset
X = X10_test
y = y10_test
print(set(y))


{1, 2, 3, 4, 5, 6, 7}


In [7]:
# The label should start from 0, but by default, the labels are from 1 to 7.
# Change them to the range 0 to 6 by subtracting 1 from the labels.
y = y-1
print(set(y))


{0, 1, 2, 3, 4, 5, 6}


In [8]:
# Split the data into 80% training and 20% testing
from sklearn.model_selection import train_test_split
# While splitting, make an unbiased splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
 stratify=y, random_state=42)
# Feature scaling using Standardization
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# Training the feature scaling parameters
sc.fit(X_train)


StandardScaler(copy=True, with_mean=True, with_std=True)

In [9]:
# Applying transformations to both training and testing set
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)
# Create neural network using keras API
# Sequential() does linear stacking of layers
model_DNN = keras.models.Sequential()
# Hidden layer definitions
model_DNN.add(keras.layers.Dense(units=25, activation='relu', input_shape=
X_train.shape[1:]))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
# Batch Normalization is applied after the first layer transformation in this example.
model_DNN.add(keras.layers.BatchNormalization())
model_DNN.add(keras.layers.Dense(units=20, activation='relu'))

In [11]:
model_DNN.add(keras.layers.Dense(units= 15, activation='relu'))
model_DNN.add(keras.layers.Dense(units=10, activation='relu'))
# Output layer definitions
model_DNN.add(keras.layers.Dense(units=7, activation='sigmoid'))
# Print the summary of network architecture
model_DNN.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 25)                275       
_________________________________________________________________
batch_normalization (BatchNo (None, 25)                100       
_________________________________________________________________
dense_1 (Dense)              (None, 20)                520       
_________________________________________________________________
dense_2 (Dense)              (None, 15)                315       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                160       
_________________________________________________________________
dense_4 (Dense)              (None, 7)                 77        
Total params: 1,447
Trainable params: 1,397
Non-trainable params: 50
_____________________________________________________

In [0]:
# loss, optimizer and metrics are three important configurations.
model_DNN.compile(loss='sparse_categorical_crossentropy',
 optimizer='adam', metrics=['accuracy'])

In [0]:
cb_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
 patience=10, restore_best_weights=True)


In [14]:
model_DNN.fit(x=X_train_std, y=y_train, validation_split=0.1,
 epochs=100, batch_size=16, callbacks=[cb_early_stopping])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 41832 samples, validate on 4649 samples
Epoch 1/100
41832/41832 [==============================] - 6s 132us/sample - loss: 0.8439 - acc: 0.6308 - val_loss: 0.6795 - val_acc: 0.7105
Epoch 2/100
41832/41832 [==============================] - 5s 121us/sample - loss: 0.7300 - acc: 0.6866 - val_loss: 0.6704 - val_acc: 0.7051
Epoch 3/100
41832/41832 [==============================] - 5s 123us/sample - loss: 0.7106 - acc: 0.6905 - val_loss: 0.6375 - val_acc: 0.7240
Epoch 4/100
41832/41832 [==============================] - 5s 121us/sample - loss: 0.6964 - acc: 0.6980 - val_loss: 0.6354 - val_acc: 0.7186
Epoch 5/100
41832/41832 [==============================] - 5s 122us/sample - loss: 0.6866 - acc: 0.7003 - val_loss: 0.6244 - val_acc: 0.7275
Epoch 6/100
41832/41832 [==============================] - 5s 118us/sample - loss: 0.6788 - acc: 0.7027 - val_loss: 0.6224 - val_acc: 0.7296
Epoch 7/100

In [15]:
test_loss, test_accuracy = model_DNN.evaluate(x=X_test_std, y=y_test)


11621/11621 [==============================] - 0s 25us/sample - loss: 0.5781 - acc: 0.7468


In [16]:
print(test_loss, test_accuracy)

0.578132525018443 0.7468376
